This file has a total of different faces with approximately 65 lighting scenes for each face (2414 faces
in all). The individual images are columns of the matrix X, where each image has been downsampled
to 32 × 32 pixels and converted into gray scale with values between 0 and 1. So the matrix is size
1024 × 2414. To important the file, use the following

In [179]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
results=loadmat('yalefaces.mat')
X=results['X']

Label the faces from 0 to 9

In [180]:
# Define the number of faces and images per face
n_faces = 10
n_images_per_face = 64

labels = np.repeat(np.arange(n_faces), n_images_per_face)
labels = labels.T
print(labels)



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 

(a) Grab the first 10 individual faces from the data set (each face has 64 lighting scenes). So you will have matrix of size 1024 × 640

In [181]:
data = X[:, 0:(n_faces * n_images_per_face)]
data = data.T

(b) From the 64 faces of each person, randomly pull out 14 faces for a test set and use the remaining 50 faces for training.

In [182]:
# dict stores train and test data
data_dict = {}

size = 14 / 64

x = data[(0 * 64):((0 + 1) * 64)]
y = labels[(0 * 64):((0 + 1) * 64)]

X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(x, y, test_size=size, random_state=42)

for i in range(1, n_faces):
    x = data[(i * 64):((i + 1) * 64)]
    y = labels[(i * 64):((i + 1) * 64)]
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=size, random_state=42)
    X_train_all = np.concatenate((X_train_all, X_train), 0)
    X_test_all = np.concatenate((X_test_all, X_test), 0)
    y_train_all = np.concatenate((y_train_all, y_train), 0)
    y_test_all = np.concatenate((y_test_all, y_test), 0)
    data_dict[i] = (X_train, y_train, X_test, y_test)



(c) Do a 20-mode PCA analysis of the face images and evaluate the classification accuracy using a linear discrimant analysis.

In [188]:
# Apply PCA to reduce the dimensionality of the data
pca = PCA(n_components=20)

print(np.shape(X_train_all))
X_pca_train = pca.fit_transform(X_train_all)

X_pca_test = pca.fit_transform(X_test_all)

print(np.shape(X_pca_train))
print(np.shape(y_train_all))


(500, 1024)
(500, 20)
(500,)


In [189]:
# Train a linear classifier
lda = LinearDiscriminantAnalysis()
lda.fit(X_pca_train, y_train_all)

# Evaluate the performance on the test set
y_pred = lda.predict(X_pca_test)
acc = accuracy_score(y_test_all, y_pred)
print(f"Accuracy for LDA: {acc:.2f}")

Accuracy for LDA: 0.41


In [190]:
# Train a linear classifier
clf = SVC()
clf.fit(X_pca_train, y_train_all)

# Evaluate the performance on the test set
y_pred = clf.predict(X_pca_test)
acc = accuracy_score(y_test_all, y_pred)
print(f"Accuracy for SVM: {acc:.2f}")


Accuracy for SVM: 0.51


In [191]:
# Train a linear classifier
clf = DecisionTreeClassifier()
clf.fit(X_pca_train, y_train_all)

# Evaluate the performance on the test set
y_pred = clf.predict(X_pca_test)
acc = accuracy_score(y_test_all, y_pred)
print(f"Accuracy for DTC: {acc:.2f}")

Accuracy for DTC: 0.15
